In [2]:
!pip install -qU torchvision

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install pillow

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install requests

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


# Pneumonia detection using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

we used this open source https://www.kaggle.com/code/dnik007/pneumonia-detection-using-pytorch notebook.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

In [6]:
import json
metadata = json.load(open("/opt/ml/metadata/resource-metadata.json","r"))
bucket = metadata["UserProfileName"]
bucket

'c3396cf2-d82e-4853-a9b9-07e8f7616a64'

In [7]:
bucket

'c3396cf2-d82e-4853-a9b9-07e8f7616a64'

In [8]:
from boto3 import client

conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

data/1.10.214.9315.99939.180760.3494081.28033633183719183053153766083_3.19.340.9010.10356.640807.6622740.50575721835623011405670548627_1.21.846.3536.75118.466218.2928595.51151775852883440650842394691.dcm
data/1.11.400.3926.34696.746482.2165088.19139580634879865325718325537_6.60.116.3996.89253.601219.9903114.91562476804718415418085171010_4.15.806.6257.13258.760967.1608804.60316827540177794174576178336.dcm
data/1.11.562.7159.11982.469604.1248021.12514684284681791197348885920_3.51.219.1259.93920.608180.1532953.10398568027056818942814728454_9.49.872.6148.85718.826644.5773493.56289519279270148505072883754.dcm
data/1.14.536.3505.92562.619945.7275639.61555304190867938148876437748_2.56.705.1855.27794.182760.9200628.71223692030840253860531038583_1.35.554.4996.59962.120859.1804713.62913470258912147280017545806.dcm
data/1.14.934.4811.16898.256606.7237887.43182774895661876408788633698_3.57.747.2074.54936.714472.9118956.10840034885768289883427064774_2.88.755.7170.38067.153766.9703146.46620957710813

In [9]:
# Downloading Data from S3

import boto3
import botocore
from boto3 import client
import os
os.mkdir("data")
os.mkdir("labels")
# os.mkdir("PNEUMONIA")
# os.mkdir("NORMAL")
BUCKET_NAME = bucket
c = 0
conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])
    KEY = key['Key']
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, f"{key['Key']}")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
    c = c+1


data/1.10.214.9315.99939.180760.3494081.28033633183719183053153766083_3.19.340.9010.10356.640807.6622740.50575721835623011405670548627_1.21.846.3536.75118.466218.2928595.51151775852883440650842394691.dcm
data/1.11.400.3926.34696.746482.2165088.19139580634879865325718325537_6.60.116.3996.89253.601219.9903114.91562476804718415418085171010_4.15.806.6257.13258.760967.1608804.60316827540177794174576178336.dcm
data/1.11.562.7159.11982.469604.1248021.12514684284681791197348885920_3.51.219.1259.93920.608180.1532953.10398568027056818942814728454_9.49.872.6148.85718.826644.5773493.56289519279270148505072883754.dcm
data/1.14.536.3505.92562.619945.7275639.61555304190867938148876437748_2.56.705.1855.27794.182760.9200628.71223692030840253860531038583_1.35.554.4996.59962.120859.1804713.62913470258912147280017545806.dcm
data/1.14.934.4811.16898.256606.7237887.43182774895661876408788633698_3.57.747.2074.54936.714472.9118956.10840034885768289883427064774_2.88.755.7170.38067.153766.9703146.46620957710813

In [71]:
# Converting downloaded data to png

# Dcm to Png
!pip install pydicom
import pydicom as dicom
import pydicom
import json
import os
from pathlib import Path
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom import dcmread
import os
import pickle
from PIL import Image
import numpy as np
from PIL import Image
import PIL

def read_xray(path, voi_lut=True, fix_monochrome=True):
    try:
        print("Converting to PNG .........................")
        dicom = dcmread(path, force=True)
        print(dicom.SOPInstanceUID, ">>>>>>", dicom.StudyInstanceUID, ">>>>>", dicom.SeriesInstanceUID)
        #if voi_lut:
        if voi_lut and len(dicom.get("VOILUTSequence", [])):
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data,dicom.PatientName
    except Exception as e:
        print(e, ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        return "corrupt"

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [73]:
from glob import glob
files_ = glob('data/*.dcm',recursive=True)
try:
    os.mkdir("training_data_png")
except:
    pass
print(files_)
c = 0
for i in files_:
    try:
        print("\n>>> ",i)
        img,patientname = read_xray(i)
        with open("filename.pkl", 'wb') as f:
            pickle.dump(img, f)
        ims = pickle.load(open("filename.pkl", "rb"))
        norm = (ims.astype(np.float) - ims.min()) * 255.0 / (ims.max() - ims.min())
        filename = str(i).split("/")[1].split("_")[0]
        print("\n>> filename",filename)
        Image.fromarray(norm.astype(np.uint8)).save(f"training_data_png/{filename}.png")
        c = c + 1
        print(c, "Done")
    except Exception as e:
        print(e)

['data/7.27.815.6722.18859.184287.9681643.15800284996471412121344288177_6.98.373.5030.76539.707169.1468683.82843168631796368677669099852_5.98.923.8675.82470.637845.6118216.20494792530915678488722155803.dcm', 'data/2.17.387.7156.15529.599631.8209533.86254490370148406471669405791_1.11.213.6495.25786.974850.7380707.16146688862807051642626781579_7.87.614.9069.59462.616722.5978389.44495173555266868499707253314.dcm', 'data/8.19.449.8327.89442.661596.6960585.56692797080442158883405395900_4.78.538.2031.93467.545861.6719955.40287632269185706961952425907_6.49.173.3991.15079.449874.5849697.27399114180762120511369333130.dcm', 'data/2.23.989.2490.19376.703142.8254473.36642501573457645052323516896_6.47.180.2992.56089.879491.6688802.34716767844240119619074427751_1.88.984.8143.41479.434574.4720560.74582485003128000639903601435.dcm', 'data/7.15.463.6329.28521.954578.3041919.39436735276558543764610736373_8.36.120.8261.70782.541687.9175246.36680293849850826756212851470_2.66.464.7338.56248.208184.7125591.

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()


1 Done

>>>  data/2.17.387.7156.15529.599631.8209533.86254490370148406471669405791_1.11.213.6495.25786.974850.7380707.16146688862807051642626781579_7.87.614.9069.59462.616722.5978389.44495173555266868499707253314.dcm
Converting to PNG .........................
7.87.614.9069.59462.616722.5978389.44495173555266868499707253314 >>>>>> 2.17.387.7156.15529.599631.8209533.86254490370148406471669405791 >>>>> 1.11.213.6495.25786.974850.7380707.16146688862807051642626781579

>> filename 2.17.387.7156.15529.599631.8209533.86254490370148406471669405791
2 Done

>>>  data/8.19.449.8327.89442.661596.6960585.56692797080442158883405395900_4.78.538.2031.93467.545861.6719955.40287632269185706961952425907_6.49.173.3991.15079.449874.5849697.27399114180762120511369333130.dcm
Converting to PNG .........................
6.49.173.3991.15079.449874.5849697.27399114180762120511369333130 >>>>>> 8.19.449.8327.89442.661596.6960585.56692797080442158883405395900 >>>>> 4.78.538.2031.93467.545861.6719955.40287632269185

## Data
### Getting the data



### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [49]:
from glob import glob
import pandas as pd
labels = pd.read_csv("./labels/labels.csv",header=1)
print(labels)


# inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
# print("input spec (in this case, just an S3 path): {}".format(inputs))

     0 Case Number  Patient ID                   Patient Name  \
0    1   Case 3340         NaN  person1946_bacteria_4874.jpeg   
1    2   Case 3342         NaN  person1947_bacteria_4876.jpeg   
2    3   Case 3354         NaN              IM-0128-0001.jpeg   
3    4   Case 3351         NaN              IM-0122-0001.jpeg   
4    5   Case 3357         NaN              IM-0133-0001.jpeg   
5    6   Case 3355         NaN              IM-0129-0001.jpeg   
6    7   Case 3364         NaN       person3_bacteria_12.jpeg   
7    8   Case 3369         NaN              IM-0003-0001.jpeg   
8    9   Case 3343         NaN  person1949_bacteria_4880.jpeg   
9   10   Case 3359         NaN        person1_bacteria_2.jpeg   
10  11   Case 3384         NaN   person1_virus_9_gOqrsNV.jpeg   
11  12   Case 3382         NaN   person1_virus_7_QW1xtMB.jpeg   
12  13   Case 3336         NaN      NORMAL2-IM-1437-0001.jpeg   
13  14   Case 3358         NaN        person1_bacteria_1.jpeg   
14  15   Case 3361       

In [80]:
files_ = glob('training_data_png/*.png',recursive=True)
# for i in labels.iterrows():
#     print(i[1][4],i[1][7])
    
#     if i[1][3].startswith("person"):
#         files_ = glob('training_data_png/'+i[1][4]+'.png',recursive=True)
#         print(files_)
#         # sagemaker_session.upload_data(path="training_data_png/" + , bucket=bucket, key_prefix=prefix)
        
import random

# all_players = list(range(12))

random.shuffle(files_)

print(files_[:len(files_)//2])
print(files_[len(files_)//2:])

normals = files_[:len(files_)//2]
abnormals = files_[len(files_)//2:]

for file in normals:
    sagemaker_session.upload_data(path=file  , bucket=bucket, key_prefix="data/train/normal")
for file in abnormals:
    sagemaker_session.upload_data(path=file  , bucket=bucket, key_prefix="data/train/abnormal")


['training_data_png/1.15.400.5298.45295.984740.2508394.92898965146452602225252758433.png', 'training_data_png/7.65.283.3257.52692.293524.3264951.88052777763521572716594030392.png', 'training_data_png/8.37.147.1346.49500.735274.3263798.93661548313874217613107411850.png', 'training_data_png/2.39.169.7400.55017.386303.4284305.19748368062157618295710183906.png', 'training_data_png/8.19.449.8327.89442.661596.6960585.56692797080442158883405395900.png', 'training_data_png/6.43.712.8966.76987.320151.2098012.68539084643889424440644432790.png', 'training_data_png/1.11.400.3926.34696.746482.2165088.19139580634879865325718325537.png', 'training_data_png/3.87.106.9932.10741.156368.3325180.37985895898340715534046907123.png', 'training_data_png/7.27.815.6722.18859.184287.9681643.15800284996471412121344288177.png', 'training_data_png/1.58.107.3370.66216.467927.3032429.87173244635710878592491214040.png', 'training_data_png/Anon.png', 'training_data_png/6.60.631.4821.31180.873857.5596272.194998800991723

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [13]:
!pygmentize code/classifier.py

import argparse
import json
import logging
import os
import sys
#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

import shlex, subprocess
# def install(package):
#     os.system("pip install " +  package)
    
# install('pillow')
# install('requests')
# install('pydicom')
from PIL import Image
import requests
import pydicom

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def get_pretrained_model(model_name):
    """Retrieve a pre-trained model from torchvision

    Params
    -------
        model_name (str): name of the model (currently only accepts vgg16 and resnet50)

    Return
    --------
        model (PyTorch model): cnn

    """

    if model_name == 'vgg16':
        model = models.vgg16(pretr

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [9]:
!pip install --upgrade sagemaker==2.110.0

In [82]:
from sagemaker.pytorch import PyTorch


In [83]:
!pip show sagemaker | grep Version

Version: 2.120.0


In [84]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="classifier.py",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=1,
    instance_type="ml.m5.large",
    # instance_type="ml.g4dn.xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
    dependencies=['code/requirements.txt'],
    source_dir = "code",
    max_run=20,
    
)

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [85]:
estimator.fit(
    {
        "training":"s3://"+bucket+"/data/train" ,
        "testing":"s3://"+bucket+"/data/train",
        "validating":"s3://"+bucket+"/data/train"
        }
    )

2023-01-31 07:48:56 Starting - Starting the training job...
2023-01-31 07:49:12 Starting - Preparing the instances for trainingProfilerReport-1675151335: InProgress
......
2023-01-31 07:50:06 Downloading - Downloading input data...
2023-01-31 07:50:51 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-01-31 02:52:36,270 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-01-31 02:52:36,272 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-31 02:52:36,273 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-01-31 02:52:36,284 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-01-31 02:52:36,300 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-01-31 02:52:36,534 sagemaker-t

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [86]:
estimator.__dict__


{'framework_version': '1.11.0',
 'py_version': 'py38',
 'role': 'arn:aws:iam::883411003189:role/SageMaker-Customer01',
 'instance_count': 1,
 'instance_type': 'ml.m5.large',
 'keep_alive_period_in_seconds': None,
 'instance_groups': None,
 'volume_size': 30,
 'volume_kms_key': None,
 'max_run': 20,
 'input_mode': 'File',
 'metric_definitions': None,
 'model_uri': None,
 'model_channel_name': 'model',
 'code_uri': None,
 'code_channel_name': 'code',
 'source_dir': 'code',
 'git_config': None,
 'container_log_level': 20,
 '_hyperparameters': {'epochs': 1,
  'backend': 'gloo',
  'sagemaker_submit_directory': 's3://sagemaker-ap-southeast-1-883411003189/pytorch-training-2023-01-31-07-48-55-035/source/sourcedir.tar.gz',
  'sagemaker_program': 'classifier.py',
  'sagemaker_container_log_level': 20,
  'sagemaker_job_name': 'pytorch-training-2023-01-31-07-48-55-035',
  'sagemaker_region': 'ap-southeast-1'},
 'code_location': None,
 'entry_point': 'classifier.py',
 'dependencies': ['code/require

In [87]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.2xlarge")

------!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [88]:
predictor.__dict__

{'endpoint_name': 'pytorch-training-2023-01-31-07-58-27-135',
 'sagemaker_session': <sagemaker.session.Session at 0x7ff528ff5bd0>,
 'serializer': <sagemaker.serializers.NumpySerializer at 0x7ff52938d050>,
 'deserializer': <sagemaker.deserializers.NumpyDeserializer at 0x7ff528ffc490>,
 '_endpoint_config_name': None,
 '_model_names': None,
 '_context': None}

In [89]:
predictor.endpoint_name 

'pytorch-training-2023-01-31-07-58-27-135'

In [90]:
import boto3

client = boto3.client('sagemaker-runtime')

custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.
endpoint_name = predictor.endpoint_name                                        # Your endpoint name.
content_type = "application/json"                                        # The MIME type of the input data in the request body.
accept = "application/json"                                              # The desired MIME type of the inference in the response.
payload = json.dumps({"url":"https://storage.googleapis.com/kaggle-datasets-images/17810/23340/c8372ebbe20b0f671c2f3c501ba51412/dataset-cover.jpeg?t=2018-03-24-19-05-18"})                                           # Payload for inference.
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes=custom_attributes, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response)   

{'ResponseMetadata': {'RequestId': '17f1099c-72b4-4c9c-b635-29936080eaff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '17f1099c-72b4-4c9c-b635-29936080eaff', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Tue, 31 Jan 2023 08:02:32 GMT', 'content-type': 'application/json', 'content-length': '66'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7ff52beb1cd0>}


In [91]:
from pprint import pprint

In [92]:
pprint(response) 

{'Body': <botocore.response.StreamingBody object at 0x7ff52beb1cd0>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '66',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 31 Jan 2023 08:02:32 GMT',
                                      'x-amzn-invoked-production-variant': 'AllTraffic',
                                      'x-amzn-requestid': '17f1099c-72b4-4c9c-b635-29936080eaff'},
                      'HTTPStatusCode': 200,
                      'RequestId': '17f1099c-72b4-4c9c-b635-29936080eaff',
                      'RetryAttempts': 0}}


In [93]:
r = json.load(response["Body"])

In [94]:
r

{'response': {'findings': [{'name': 'class', 'probability': 62}]}}